In [1]:
from atmospheric_explorer.cams_interfaces import InversionOptimisedGreenhouseGas, EAC4Instance
from atmospheric_explorer.shapefile import ShapefilesDownloader
from atmospheric_explorer.utils import get_local_folder
from atmospheric_explorer.units_conversion import convert_units_array
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os
from glob import glob
import xarray as xr
from datetime import datetime
import geopandas as gpd
from shapely.geometry import mapping
import plotly.graph_objects as go
import shutil
import numpy as np
import statsmodels.stats.api as sms
import pandas as pd
from plotly.subplots import make_subplots
from math import ceil
from atmospheric_explorer.plotting_apis import ghg_surface_satellite_yearly_plot
from atmospheric_explorer.ui.interactive_map.shape_selection import ShapeSelection

2023-08-19 18:25:16,744 INFO Loaded constants from file constants.cfg
2023-08-19 18:25:16,744 INFO Evaluating arithmetic formulas in config
2023-08-19 18:25:16,745 DEBUG Parsing 1
2023-08-19 18:25:16,745 DEBUG Parsing 1
2023-08-19 18:25:16,746 DEBUG Parsing 1
2023-08-19 18:25:16,746 DEBUG Parsing 1
2023-08-19 18:25:16,746 DEBUG Parsing 1
2023-08-19 18:25:16,747 DEBUG Parsing 1
2023-08-19 18:25:16,747 DEBUG Parsing 1
2023-08-19 18:25:16,747 DEBUG Parsing 1
2023-08-19 18:25:16,748 DEBUG Parsing 1
2023-08-19 18:25:16,748 DEBUG Parsing 1
2023-08-19 18:25:16,748 DEBUG Parsing 1
2023-08-19 18:25:16,748 DEBUG Parsing 1
2023-08-19 18:25:16,749 DEBUG Parsing 1
2023-08-19 18:25:16,749 DEBUG Parsing 1
2023-08-19 18:25:16,749 DEBUG Parsing 1
2023-08-19 18:25:16,749 DEBUG Parsing 1
2023-08-19 18:25:16,749 DEBUG Parsing 1
2023-08-19 18:25:16,750 DEBUG Parsing 1
2023-08-19 18:25:16,750 DEBUG Parsing 1
2023-08-19 18:25:16,750 DEBUG Parsing 1
2023-08-19 18:25:16,750 DEBUG Parsing 1
2023-08-19 18:25:16,

In [2]:
# Remove previous data
shutil.rmtree(os.path.join(get_local_folder(), 'data'))

In [3]:
# Remove previous data
shutil.rmtree(os.path.join(get_local_folder(), 'shapefiles'))

In [4]:
years=[str(y) for y in range(2018, 2021)]
months=[
    '01', '02', '03',
    '04', '05', '06',
    '07', '08', '09',
    '10', '11', '12'
]
countries = ['Italy']
data_variable = 'carbon_dioxide'
var_name = 'flux_foss'

In [5]:
shapes = ShapeSelection.from_countries_list(countries)

2023-08-19 18:25:19.050 
  command:

    streamlit run /home/luigi/miniconda3/envs/atmospheric-explorer/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-08-19 18:25:19.050 No runtime found, using MemoryCacheStorageManager
2023-08-19 18:25:19,051 DEBUG Created ShapefilesDownloader object with attributes
dst_dir: /home/luigi/.atmospheric_explorer/shapefiles
resolution: 10m
info_type: admin
depth: 0
instance: countries

2023-08-19 18:25:19,051 INFO Created folder /home/luigi/.atmospheric_explorer/shapefiles to save shapefiles
2023-08-19 18:25:19,052 INFO Shapefile not found, downloading it from Natural Earth Data
2023-08-19 18:25:19,052 INFO Downloading shapefiles from https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_0_countries.zip
2023-08-19 18:25:20,759 INFO Shapefiles downloaded to file /home/luigi/.atmospheric_explorer/shapefiles/ne_10m_admin_0_countries.zip
2023-08-19 18:25:20,797 INFO Shapefile extracted to /home/

In [6]:
fig = ghg_surface_satellite_yearly_plot(
    data_variable,
    years,
    months,
    "Fossil CO2 flux",
    var_name,
    shapes=shapes.dataframe,
    add_satellite_observations=False
)
fig.show()

2023-08-19 18:25:21,954 DEBUG     ghg_surface_satellite_yearly_plot called with arguments
    data_variable: carbon_dioxide
    shapes:     label                                           geometry
90  Italy  MULTIPOLYGON (((7.02208 45.92526, 7.06694 45.8...
    years: ['2018', '2019', '2020']
    months: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    title: Fossil CO2 flux
    var_name: flux_foss

2023-08-19 18:25:21,954 INFO Created folder /home/luigi/.atmospheric_explorer/data/global_greenhouse_gas_inversion
2023-08-19 18:25:21,955 INFO Created folder /home/luigi/.atmospheric_explorer/data/global_greenhouse_gas_inversion/data_1
2023-08-19 18:25:22,147 INFO Welcome to the CDS
2023-08-19 18:25:22,149 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-greenhouse-gas-inversion
2023-08-19 18:25:22,218 INFO Request is queued
2023-08-19 18:25:23,278 INFO Request is completed
2023-08-19 18:25:23,278 INFO Downloading https:/

In [ ]:
data_variable = "total_column_nitrogen_dioxide"
var_name = "tcno2"
countries = ["Portugal"]
dates_range = "2018-01-01/2020-01-01"
time_values = "00:00"
title = "Total columns N02 above Portugal (anomalies)"

In [ ]:
fig = eac4_anomalies_plot(
    data_variable,
    var_name,
    countries,
    dates_range,
    time_values,
    title,
)
fig.show()

In [ ]:
data_variable = 'total_column_ozone'
var_name = 'gtco3'
dates_range = '2020-01-01/2022-12-31'
time_values = '00:00'
title = 'O3 total column'

In [ ]:
eac4_hovmoeller_latitude_plot(
    data_variable,
    var_name,
    dates_range,
    time_values,
    title,
)

In [ ]:
data_variable = 'carbon_monoxide'
var_name = 'co'
dates_range = '2022-01-01/2022-12-31'
time_values = '00:00'
title = 'CO'

In [ ]:
eac4_hovmoeller_levels_plot(
    data_variable,
    var_name,
    dates_range,
    time_values,
    ['1', '10', '100'],
    countries,
    title,
)

In [ ]:
import plotly.express as px

In [ ]:
fig = px.imshow(
    img = [[1,2], [3,4]],
    x = ['A', 'B'],
    y = [1, 2],
    text_auto=True
)
fig.update_yaxes(type='log')

In [ ]:
from atmospheric_explorer.plotting_apis import ghg_surface_satellite_yearly_plot

data_variable = 'carbon_dioxide'
var_name = ['flux_apos_bio', 'flux_apos']
countries = ['Italy']
years = ['2020', '2021']
months = ['01', '02', '03', '04']
add_satellite_observations = False
plot_title = 'Test'

fig = ghg_surface_satellite_yearly_plot(
    data_variable=data_variable,
    var_name=var_name,
    add_satellite_observations=add_satellite_observations,
    countries=countries,
    years=years,
    months=months,
    title=plot_title,
)

display(fig)